In [6]:
import cv2
import os
import numpy as np
from datetime import datetime
import pandas as pd
import time
from deepface import DeepFace

# Extract student names from dataset 
def get_enrolled_students(db_path="C:/Users/Anshika/Downloads/emotion_detection/students"):
    names = set()
    for root, _, files in os.walk(db_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                name = os.path.basename(file).split('.')[0]
                names.add(name)
    return list(names)

# Time check function
def in_class_time():
    now = datetime.now().time()
    return datetime.strptime("9:30", "%H:%M").time() <= now <= datetime.strptime("10:00", "%H:%M").time()

# Face recognition
def identify_student(frame_path):
    try:
        result = DeepFace.find(img_path=frame_path, db_path="C:/Users/Anshika/Downloads/emotion_detection/students", enforce_detection=False)
        if result and not result[0].empty:
            return os.path.basename(result[0].iloc[0]['identity']).split('.')[0]
    except:
        pass
    return None

# Emotion detection
def detect_emotion(frame_path):
    try:
        result = DeepFace.analyze(img_path=frame_path, actions=['emotion'], enforce_detection=False)
        return result[0]['dominant_emotion']
    except:
        return "Unknown"

# Saving attendance of all the students
def save_attendance(records, present_students, enrolled_students):
    all_records = []
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for student in enrolled_students:
        if student in present_students:
            for rec in records:
                if rec[0] == student:
                    all_records.append(rec)
                    break
        else:
            all_records.append([student, "Absent", "N/A", timestamp])
    df = pd.DataFrame(all_records, columns=["Name", "Status", "Emotion", "Timestamp"])
    df.to_csv("attendance.csv", index=False)
    print("Attendance saved to attendance.csv")

# Main attendance function
def run_attendance():
    enrolled_students = get_enrolled_students("C:/Users/Anshika/Downloads/emotion_detection/students")
    cap = cv2.VideoCapture(0)
    present_students = set()
    records = []

    print("Attendance system started...")

    while in_class_time():
        ret, frame = cap.read()
        if not ret:
            continue
        frame_path = "temp_frame.jpg"
        cv2.imwrite(frame_path, frame)

        student = identify_student(frame_path)
        if student and student not in present_students:
            emotion = detect_emotion(frame_path)
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            records.append([student, "Present", emotion, timestamp])
            present_students.add(student)
            print(f"{student} marked Present with emotion: {emotion} at {timestamp}")

        time.sleep(1)

    cap.release()
    save_attendance(records, present_students, enrolled_students)
    print("Attendance session ended.")

# Execution of the script
if __name__ == "__main__":
    if in_class_time():
        run_attendance()
    else:
        print("Current time is outside the attendance window (09:30–10:00 AM).")

Attendance system started...
25-06-25 15:03:50 - Searching temp_frame.jpg in 11 length datastore
25-06-25 15:03:51 - find function duration 0.9764277935028076 seconds
25-06-25 15:03:52 - Searching temp_frame.jpg in 11 length datastore
25-06-25 15:03:52 - find function duration 0.7489268779754639 seconds
25-06-25 15:03:53 - Searching temp_frame.jpg in 11 length datastore
25-06-25 15:03:54 - find function duration 0.8112509250640869 seconds
Anshi marked Present with emotion: fear at 2025-06-25 15:03:54
25-06-25 15:03:56 - Searching temp_frame.jpg in 11 length datastore
25-06-25 15:03:56 - find function duration 0.8068575859069824 seconds
25-06-25 15:03:57 - Searching temp_frame.jpg in 11 length datastore
25-06-25 15:03:58 - find function duration 0.8035888671875 seconds
25-06-25 15:03:59 - Searching temp_frame.jpg in 11 length datastore
25-06-25 15:04:00 - find function duration 0.7423546314239502 seconds
Attendance saved to attendance.csv
Attendance session ended.
